In [1]:
import pandas as pd
import numpy as np
from dateutil import relativedelta
pd.options.display.max_rows = 100
from datetime import datetime
import re
from itertools import combinations, product



In [2]:
from os import pardir, path
import sys
mod_path = path.abspath(path.join(pardir))

print(mod_path)

if mod_path not in sys.path:
    sys.path.append(mod_path)

print(sys.path)
from data_extract import client, fin, company , companyfinancials_list, pnl, bs, cf, profile, prices, fx, mrktcap, prices_n_fin, idx_price, MONGO_FIN

/Users/karimkhalil/Coding/development/fin
['/Users/karimkhalil/Coding/development/fin/notebooks', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python39.zip', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python3.9', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python3.9/lib-dynload', '', '/Users/karimkhalil/Coding/development/fin/conda_env/lib/python3.9/site-packages', '/Users/karimkhalil/Coding/development/fin']
['admin', 'config', 'db_fin', 'local']
['cf', 'prices_bckup', 'company_today', 'mrktcap', 'company_wfinancials', 'prices_n_fin', 'bs', 'fx', 'profile', 'idx_price', 'prices', 'idx_all', 'pnl']


In [3]:
db = MONGO_FIN()

In [4]:
import json
mapping = json.load(open("../config/config.json"))['mapping']

#### Currency data

In [5]:
df_fx1 = db.get_df(fx)
df_fx2 = df_fx1[['symbol' , 'adjClose', 'date']]
df_fx2['datetime'] = pd.to_datetime(df_fx2.date, format='%Y-%m-%d')
df_fx2['yearmonth'] = df_fx2['datetime'].apply(lambda x: x.strftime('%Y%m'))
df_fx2['year'] = df_fx2['datetime'].apply(lambda x: x.strftime('%Y'))
df_fx2['month'] = df_fx2['datetime'].apply(lambda x: x.strftime('%m'))
df_fx2.symbol = df_fx2.symbol.str.replace("/USD", "")

## DataFrame for the average fx for a given month ==> to be used to convert the fx for stock prices and market cap
df_fx3 = df_fx2.groupby(['symbol' , 'yearmonth' , 'year' , 'month'])['adjClose'].mean().reset_index()
df_fx3['key_fx'] = df_fx3.symbol+"."+df_fx3.yearmonth.astype(str)

/var/folders/_7/8tv1x9sn12bcg2fzw5kp9n7h0000gn/T/ipykernel_78507/1913243675.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fx2['datetime'] = pd.to_datetime(df_fx2.date, format='%Y-%m-%d')
/var/folders/_7/8tv1x9sn12bcg2fzw5kp9n7h0000gn/T/ipykernel_78507/1913243675.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fx2['yearmonth'] = df_fx2['datetime'].apply(lambda x: x.strftime('%Y%m'))
/var/folders/_7/8tv1x9sn12bcg2fzw5kp9n7h0000gn/T/ipykernel_78507/1913243675.py:5: SettingWithCopyWarning: 
A va

In [6]:
## financials FX ==> calculate fx for last 3 months
df_fx4 = df_fx3
fx_periods = list(df_fx4.yearmonth.unique())
fx_periods2 = [i for i in fx_periods]
fx_periods3 = {i: [(datetime.strptime(i, "%Y%m") + relativedelta.relativedelta(months=-j)).strftime('%Y%m')for j in range(3)] for i in fx_periods2}

## DataFrame for the average fx for a given quarter ==> to be used to convert the fx for financials
df_fx4['previous_q'] = df_fx4.apply(lambda row: fx_periods3[row['yearmonth']] , axis =1)
df_fx4['previous_q'] = df_fx4.apply(lambda row: [row['symbol']+"."+ str(i) for i in row['previous_q']] , axis =1)
df_fx4['previous_q_fx'] =  df_fx4.apply(lambda row: df_fx4.loc[df_fx4['key_fx'].isin(row['previous_q']), "adjClose"].mean() , axis =1)

#### Stock data

In [7]:
# import csv
df_stock_mrkt1 = pd.read_csv('../data/df_stock_mrkt_q.csv')

In [8]:
# merge fx with stock data

df_stock_mrkt2 = df_stock_mrkt1.copy()
df_stock_mrkt2.currency = df_stock_mrkt2.currency.str.upper()
df_stock_mrkt2.insert(5, 'datetime' , value = pd.to_datetime(df_stock_mrkt2.date, format='%Y-%m-%d'))
df_stock_mrkt2.insert(5, column='yearmonth', value=df_stock_mrkt2['datetime'].apply(lambda x: x.strftime('%Y%m')))
df_stock_mrkt2.insert(5, column='year', value=df_stock_mrkt2['datetime'].apply(lambda x: x.strftime('%Y')))
df_stock_mrkt2.insert(5, column='month', value=df_stock_mrkt2['datetime'].apply(lambda x: x.strftime('%m')))
df_stock_mrkt2.insert(5, column='key_fx', value=df_stock_mrkt2.currency+"."+df_stock_mrkt2.yearmonth.astype(str))
df_stock_mrkt2.insert(5, column='fx_stock', value=df_stock_mrkt2.merge(df_fx3[['key_fx' , 'adjClose']], how = 'left', on = 'key_fx')['adjClose_y'])
df_stock_mrkt2.insert(5, column='fx_quarter', value=df_stock_mrkt2.merge(df_fx4[['key_fx' , 'previous_q_fx']], how = 'left', on = 'key_fx')['previous_q_fx'])
df_stock_mrkt2.loc[df_stock_mrkt2.currency == 'USD' , 'fx_stock'] = 1
df_stock_mrkt2.loc[df_stock_mrkt2.currency == 'USD' , 'fx_quarter'] = 1

In [9]:
quarter = {
    1: "Q1", 
    2: "Q1", 
    3: "Q1", 
    4: "Q2", 
    5: "Q2", 
    6: "Q2", 
    7: "Q3", 
    8: "Q3",
    9: "Q3",
    10: "Q4", 
    11: "Q4",
    12: "Q4"
}

In [10]:
df_stock_mrkt2.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839749 entries, 0 to 839748
Data columns (total 148 columns):
 #    Column                                       Non-Null Count   Dtype         
---   ------                                       --------------   -----         
 0    Unnamed: 0                                   839749 non-null  int64         
 1    key                                          839749 non-null  object        
 2    symbol                                       839749 non-null  object        
 3    companyName                                  839688 non-null  object        
 4    currency                                     838132 non-null  object        
 5    fx_quarter                                   827024 non-null  float64       
 6    fx_stock                                     827024 non-null  float64       
 7    key_fx                                       838132 non-null  object        
 8    month                                        839749 

In [11]:
# normalize common attributes from financials
df_stock_mrkt2['fin_reportedCurrency'] = df_stock_mrkt2['bs_reportedCurrency']
df_stock_mrkt2['fin_fillingDate'] = df_stock_mrkt2['bs_fillingDate']
df_stock_mrkt2['fin_acceptedDateDate'] = df_stock_mrkt2['bs_acceptedDate']
df_stock_mrkt2['fin_calendarYear'] = df_stock_mrkt2['bs_calendarYear']
df_stock_mrkt2['fin_period'] = df_stock_mrkt2['bs_period']
df_stock_mrkt2['cal_period'] = df_stock_mrkt2['month'].apply(lambda x: quarter[int(x)])
df_stock_mrkt2['cal_yearperiod'] = df_stock_mrkt2['year'].astype(str) + df_stock_mrkt2['cal_period']
df_stock_mrkt2['key_main'] = df_stock_mrkt2['symbol'] + "_" + df_stock_mrkt2['cal_yearperiod']

In [12]:
# industry categorization mapping
mapping = json.load(open("../config/config.json"))['mapping']
df_stock_mrkt2['industry-category'] = df_stock_mrkt2.industry.map(mapping['industry'])
df_industries = df_stock_mrkt2.groupby(['industry' , 'industry-category'])['symbol'].count().reset_index()

In [13]:
## industry mapping checks
df_industries = df_stock_mrkt2.groupby(['symbol' , 'industry'])['adjClose'].count().reset_index().drop(columns='adjClose')
mapping = json.load(open("../config/config.json"))['mapping']
df_industry_category = df_industries.industry.value_counts().reset_index()
df_industry_category.columns = ['industry' , 'industry_count']
df_industry_category['industry_category'] = df_industry_category.industry.map(mapping['industry'])

In [14]:
# relevant columns for data
cols_stock = json.load(open("../config/config.json"))['cols']['stock']
df_stock_mrkt3 = df_stock_mrkt2[cols_stock]

In [15]:
df_stock_mrkt3.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839749 entries, 0 to 839748
Data columns (total 125 columns):
 #    Column                                       Non-Null Count   Dtype         
---   ------                                       --------------   -----         
 0    key                                          839749 non-null  object        
 1    key_main                                     839749 non-null  object        
 2    symbol                                       839749 non-null  object        
 3    companyName                                  839688 non-null  object        
 4    currency                                     838132 non-null  object        
 5    fx_quarter                                   827024 non-null  float64       
 6    fx_stock                                     827024 non-null  float64       
 7    key_fx                                       838132 non-null  object        
 8    month                                        839749 

In [16]:
# remove missing data

filter_mrktcap = df_stock_mrkt3['marketCap'].notna() ## remove companies with no market capitalization 
filter_industry = df_stock_mrkt3['industry-category'].notna() ## remove some industries 
filter_fx1 = df_stock_mrkt3['fx_quarter'].notna()
filter_fx2 = df_stock_mrkt3['fx_stock'].notna()
filter_zerocash = df_stock_mrkt3['bs_cashAndShortTermInvestments'] > 0
filter_noshell  = df_stock_mrkt3['industry-category']!= 'shell' # exclude holding companies


df_stock_mrkt4 = df_stock_mrkt3.loc[filter_fx1 & filter_fx2 & filter_mrktcap & filter_industry & filter_fx1 & filter_fx2 & filter_noshell]

df_stock_mrkt4.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 612917 entries, 0 to 839748
Data columns (total 125 columns):
 #    Column                                       Non-Null Count   Dtype         
---   ------                                       --------------   -----         
 0    key                                          612917 non-null  object        
 1    key_main                                     612917 non-null  object        
 2    symbol                                       612917 non-null  object        
 3    companyName                                  612917 non-null  object        
 4    currency                                     612917 non-null  object        
 5    fx_quarter                                   612917 non-null  float64       
 6    fx_stock                                     612917 non-null  float64       
 7    key_fx                                       612917 non-null  object        
 8    month                                        612917 

In [17]:
## remove companies with more than 1 financial in a given quarter
df_stock_mrkt4.drop_duplicates(subset='key_main' , keep='last', inplace = True)
df_stock_mrkt4.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 611305 entries, 0 to 839748
Data columns (total 125 columns):
 #    Column                                       Non-Null Count   Dtype         
---   ------                                       --------------   -----         
 0    key                                          611305 non-null  object        
 1    key_main                                     611305 non-null  object        
 2    symbol                                       611305 non-null  object        
 3    companyName                                  611305 non-null  object        
 4    currency                                     611305 non-null  object        
 5    fx_quarter                                   611305 non-null  float64       
 6    fx_stock                                     611305 non-null  float64       
 7    key_fx                                       611305 non-null  object        
 8    month                                        611305 

/Users/karimkhalil/Coding/development/fin/conda_env/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [18]:
# fx translation

cols_fxspot = json.load(open("../config/config.json"))['cols']['fx_stock']
cols_fxq = json.load(open("../config/config.json"))['cols']['fx_q']

def fx(df, cols, col_fx, drop=True):
    df2 = df.copy()
    for i in cols:
        loc = df.columns.get_loc(i)
        newcol = "usd_"+i
        df2.insert(loc, newcol, df[i] * df[col_fx])
        if drop==True:
            df2.drop(columns = i, inplace=True)
    return df2

df_stock_mrkt5 = df_stock_mrkt4.copy()
df_stock_mrkt6 = df_stock_mrkt4.copy()

df_stock_mrkt5 = fx(df_stock_mrkt5, cols_fxspot, 'fx_stock', False)
df_stock_mrkt5 = fx(df_stock_mrkt5, cols_fxq, 'fx_quarter', False)

df_stock_mrkt6 = fx(df_stock_mrkt6, cols_fxspot, 'fx_stock', True)
df_stock_mrkt6 = fx(df_stock_mrkt6, cols_fxq, 'fx_quarter', True)

In [19]:
df_stock_mrkt6.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 611305 entries, 0 to 839748
Data columns (total 125 columns):
 #    Column                                           Non-Null Count   Dtype         
---   ------                                           --------------   -----         
 0    key                                              611305 non-null  object        
 1    key_main                                         611305 non-null  object        
 2    symbol                                           611305 non-null  object        
 3    companyName                                      611305 non-null  object        
 4    currency                                         611305 non-null  object        
 5    fx_quarter                                       611305 non-null  float64       
 6    fx_stock                                         611305 non-null  float64       
 7    key_fx                                           611305 non-null  object        
 8    month       

In [20]:
# df_stock_mrkt5.to_csv('../data/data_prefeatures_wlocalfx.csv')
# df_stock_mrkt6.to_csv('../data/data_prefeatures.csv')